In [1]:
!pip install '../input/pycocotools20/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl'
!pip install '../input/pytorchzoomaster/pytorch_zoo-master'
!pip install '../input/hpacellsegmentationmaster/HPA-Cell-Segmentation-master' --no-deps
!pip install '../input/efficientnetpytorch'

Processing /kaggle/input/pycocotools20/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/pytorchzoomaster/pytorch_zoo-master
  Created wheel for pytorch-zoo: filename=pytorch_zoo-0.0.0-py3-none-any.whl size=30139 sha256=85da6cf22fbe427dd6277b0da8336da6b605d10d61bffd5e9a6eb55829ba49c2
  Stored in directory: /root/.cache/pip/wheels/71/36/1d/4e2a0ae6a85f0d674d74d871538c18856f7918f67b3dd35b13
Successfully built pytorch-zoo
Processing /kaggle/input/hpacellsegmentationmaster/HPA-Cell-Segmentation-master
  Created wheel for hpacellseg: filename=hpacellseg-0.1.8-py3-none-any.whl size=14917 sha256=f2ec973e8606327d1acfdd375581b6696213a55e2ba51d05dcb8a071459e6f11
  Stored in directory: /root/.cache/pip/wheels/4c/74/b4/e1de1965e419415f68a0098b1bf022570db6548a5ef09a79ed
Successfully built hpacellseg
Processing /kaggle/input/efficientnetpytorch
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=20608 sha256=b1d0068e6b9aa98ff4e8a550a

In [2]:
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei
import numpy as np

import multiprocessing
import multiprocessing as mp

from efficientnet_pytorch import EfficientNet
import torch.nn as nn
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import pickle
import csv
import glob
import os

import base64
from pycocotools import _mask as coco_mask
import typing as t
import zlib
import ast
import cv2

import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = 512
CPU_COUNT = mp.cpu_count()

In [3]:
test_dir = '../input/hpa-single-cell-image-classification/test/'

!mkdir -p /kaggle/temp

test_cells = '/kaggle/temp/test_cells/'
!mkdir -p /kaggle/temp/test_cells

test_nuclei = '/kaggle/temp/test_nuclei/'
!mkdir -p /kaggle/temp/test_nuclei

nuclei_512 = '/kaggle/temp/nuclei_512/'
!mkdir -p /kaggle/temp/nuclei_512

cells_512 = '/kaggle/temp/cells_512/'
!mkdir -p /kaggle/temp/cells_512

NUC_MODEL = "../input/hpasegweights/dpn_unet_nuclei_v1.pth"
CELL_MODEL = "../input/hpasegweights/dpn_unet_cell_3ch_v1.pth"

segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device="cuda",
    padding=True,
    multi_channel_model=True,
)

please compile abn


/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'pytorch_zoo.unet.DPNUnet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set

In [4]:
def create_masks(cell_segmentation, nuc_segmentation, name):
    nuclei_mask, cell_mask = label_cell(nuc_segmentation, cell_segmentation)
    
    ID = name.replace('_red.png','').split('/')[-1]
    if cell_mask.max() < 256:
        np.save(test_nuclei + ID + '.npy', nuclei_mask.astype(np.uint8))
        np.save(test_cells + ID + '.npy', cell_mask.astype(np.uint8))
    else:
        np.save(test_nuclei + ID + '.npy', nuclei_mask)
        np.save(test_cells + ID + '.npy', cell_mask)
            
    return

In [5]:
mt = glob.glob(test_dir + '*_red.png')
er = [f.replace('red', 'yellow') for f in mt]
nu = [f.replace('red', 'blue') for f in mt]

In [6]:
batch = 32
pos = 0
start_time = time.time()
while pos < len(mt):

    images = [mt[pos:pos+batch], er[pos:pos+batch], nu[pos:pos+batch]]
    pos += batch

    nuc_segmentations = segmentator.pred_nuclei(images[2])
    cell_segmentations = segmentator.pred_cells(images)

    pool = mp.Pool(mp.cpu_count())

    for i in range(len(cell_segmentations)):
        name = images[0][i]
        pool.apply_async(create_masks, args=(cell_segmentations[i], nuc_segmentations[i], name))
        
    pool.close()    
    pool.join()
    print(pos, end = ' ')
elapsed_time = time.time() - start_time
print(elapsed_time / 60.0)

32 64 96 128 160 192 224 256 288 320 352 384 416 448 480 512 544 576 56.733264124393465


In [7]:
def encode_binary_mask(mask):

    # check input mask --
    if mask.dtype != np.bool:
        raise ValueError("encode_binary_mask expects a binary mask, received dtype == %s" % mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError("encode_binary_mask expects a 2d mask, received shape == %s" % mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str

In [8]:
def get_paddings(x_len, y_len):

    pad_left, pad_right, pad_up, pad_down = 0, 0, 0, 0

    if x_len > y_len:
        pad_left = (x_len-y_len)//2
        pad_right = x_len-y_len - pad_left
    else:
        pad_up = (y_len-x_len)//2
        pad_down = y_len-x_len - pad_up

    return pad_left, pad_right, pad_up, pad_down

def load_this_image(image_path):
    image = np.array(cv2.imread(image_path, cv2.IMREAD_UNCHANGED))
    
    if image.dtype == np.uint8:
        image = image.astype(np.uint16)
        image *= 257
    
    return image

def get_cropped_channel_image_nuclei(image, top_left, bottom_right, mask, cell_id):
    
    image = image[top_left[0]:bottom_right[0]+1,top_left[1]:bottom_right[1]+1].copy()
    image[(mask != 0) * (mask != cell_id)] = 0

    pad_left, pad_right, pad_up, pad_down = get_paddings(image.shape[0], image.shape[1])
    image = np.pad(image, [(pad_up, pad_down), (pad_left, pad_right)], mode='constant')

    return image

def get_cropped_channel_image_cells(image, top_left, bottom_right, mask, cell_id):
    
    image = image[top_left[0]:bottom_right[0]+1,top_left[1]:bottom_right[1]+1].copy()
    image[mask != cell_id] = 0

    pad_left, pad_right, pad_up, pad_down = get_paddings(image.shape[0], image.shape[1])
    image = np.pad(image, [(pad_up, pad_down), (pad_left, pad_right)], mode='constant')

    return image

def get_mask_info(mask):
    true_points = np.argwhere(mask)
    
    if not true_points.any():
        return np.array([0, 0]), np.array([0, 0])
    
    top_left = true_points.min(axis=0)
    bottom_right = true_points.max(axis=0)

    return top_left, bottom_right

def solve_nuclei(name):
    
    mask_dir = test_nuclei
    
    ID = name.replace('_red.png','').split('/')[-1]
    
    cell_mask = np.load(mask_dir + ID + '.npy')
    max_id = cell_mask.max()
    
    image_per_channel = {}

    for channel in ['_red', '_blue', '_green']:

        image_filename = ID + channel + '.png'
        image = load_this_image(test_dir + image_filename)

        image_per_channel[channel] = image
    
    for curr_id in range(1, max_id+1):
        
        cell_info = get_mask_info(cell_mask == curr_id)
        
        top_left, bottom_right = cell_info[0], cell_info[1]
        
        dim1 = bottom_right[0]+1 - top_left[0]
        dim2 = bottom_right[1]+1 - top_left[1]

        pad_left, pad_right, pad_up, pad_down = get_paddings(dim1, dim2)
        
        top_left[1] -= pad_left + 16
        bottom_right[1] += pad_right + 16
        top_left[0] -= pad_up + 16
        bottom_right[0] += pad_down + 16

        top_left[0] = max(0, top_left[0])
        top_left[1] = max(0, top_left[1])
        bottom_right[0] = min(cell_mask.shape[0], bottom_right[0])
        bottom_right[1] = min(cell_mask.shape[1], bottom_right[1])
        
        for channel in ['_red', '_blue', '_green']:
            save_path = nuclei_512 + ID + '_' + str(curr_id-1) + channel + '.png'
        
            big_image = image_per_channel[channel]
            image = get_cropped_channel_image_nuclei(big_image, cell_info[0], cell_info[1], cell_mask[top_left[0]:bottom_right[0]+1,top_left[1]:bottom_right[1]+1], curr_id)
            
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            cv2.imwrite(save_path, image)
            
    return

def solve_cells(name):
    
    masks = []
    
    mask_dir = test_cells
    
    ID = name.replace('_red.png','').split('/')[-1]
    
    cell_mask = np.load(mask_dir + ID + '.npy')
    max_id = cell_mask.max()
    
    h = cell_mask.shape[0]
    w = cell_mask.shape[1]
    
    image_per_channel = {}

    for channel in ['_red', '_blue', '_green', '_yellow']:

        image_filename = ID + channel + '.png'
        image = load_this_image(test_dir + image_filename)

        image_per_channel[channel] = image
    for curr_id in range(1, max_id+1):
        
        binary_mask = cell_mask == curr_id
        
        masks.append(str(encode_binary_mask(binary_mask.astype(np.bool)))[2:-1])
        
        cell_info = get_mask_info(binary_mask)
        
        top_left, bottom_right = cell_info[0], cell_info[1]
        
        for channel in ['_red', '_blue', '_green', '_yellow']:
        
            save_path = cells_512 + ID + '_' + str(curr_id-1) + channel + '.png'
        
            big_image = image_per_channel[channel]
            
            image = get_cropped_channel_image_cells(big_image, cell_info[0], cell_info[1], cell_mask[top_left[0]:bottom_right[0]+1,top_left[1]:bottom_right[1]+1], curr_id)
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            cv2.imwrite(save_path, image)
            
    return ID, masks, w, h

In [9]:
start_time = time.time()
pool = mp.Pool(CPU_COUNT)
cell_mask_infos = []

for x in mt:
    pool.apply_async(solve_cells, args=(x,), callback=cell_mask_infos.append)
    
pool.close()    
pool.join()
elapsed_time = time.time() - start_time
print(elapsed_time / 60.0)

11.042728678385417


In [10]:
mask_encodings = {}
mask_w_h = {}
for mask_info in cell_mask_infos:
    mask_encodings[mask_info[0]] = mask_info[1]
    mask_w_h[mask_info[0]] = (mask_info[2], mask_info[3])

In [11]:
def get_mask_bounding_box(mask):
    true_points = np.argwhere(mask)
    
    if not true_points.any():
        return np.array([0, 0]), np.array([0, 0])
    
    top_left = true_points.min(axis=0)
    bottom_right = true_points.max(axis=0)

    return top_left, bottom_right

def get_cropped_mask(mask, cell_id):
    top_left, bottom_right = get_mask_bounding_box((mask == cell_id))

    mask = mask[top_left[0]:bottom_right[0]+1,top_left[1]:bottom_right[1]+1].copy()
    mask[mask != cell_id] = 0
    mask[mask != 0] = 1

    return mask

def ratio_to_value_multiplier(ratio):
    if ratio > 0.3:
        return 1.0
    
    if ratio > 0.01:
        return (ratio * 3.33) ** 0.1

    return 0.001

def no_nuclei_and_border_cleaner(nuclei_mask, cell_mask, red, blue, yellow, values):
    for i in range(len(values)):
        index = i+1

        if blue[nuclei_mask == index].sum() == 0:
            values[i] *= 0.01
            continue
    
        single_nuclei_mask = get_cropped_mask(nuclei_mask, index)
        
        lr = single_nuclei_mask.shape[0]
        ud = single_nuclei_mask.shape[1]

        up = single_nuclei_mask[0].sum()
        down = single_nuclei_mask[-1].sum()
        left = single_nuclei_mask[:, -0].sum()
        right = single_nuclei_mask[:, -1].sum()

        ud_ratio = (ud - max(up, down)) / ud
        lr_ratio = (lr - max(left, right)) / lr

        values[i] *= ratio_to_value_multiplier(ud_ratio) * ratio_to_value_multiplier(lr_ratio)

    return values

def no_blue_yellow_red_cleaner(nuclei_mask, cell_mask, red, blue, yellow, values):

    red_sums = []
    blue_yellow_products = []
    valid_candidates = 0

    for i in range(len(values)):
        index = i+1

        if values[i] <= 0.01:
            red_sums.append(0)
            blue_yellow_products.append(0)
            continue
        
        valid_candidates += 1

        red_sum = red[cell_mask == index].sum()
        blue_sum = blue[cell_mask == index].sum()
        yellow_sum = yellow[cell_mask == index].sum()

        red_sums.append(red_sum)
        blue_yellow_products.append(blue_sum * yellow_sum)
    
    red_minimum = sum(red_sums) / valid_candidates * 0.077
    blue_yellow_products_minimum = sum(blue_yellow_products) / valid_candidates * 0.05

    for i in range(len(values)):
        index = i+1

        if values[i] <= 0.01:
            continue
        
        if red_sums[i] < red_minimum or blue_yellow_products[i] < blue_yellow_products_minimum:
            values[i] *= 0.001
    
    return values

class DataCleaner():

    def __init__(self):
        self.cleaners = [no_nuclei_and_border_cleaner, no_blue_yellow_red_cleaner]

    def load_image(self, image_path):
        image = np.array(cv2.imread(image_path, cv2.IMREAD_UNCHANGED))

        if image.dtype == np.uint8:
            image = image.astype(np.uint16)
            image *= 257
        
        image = image.astype(np.float32)
        image /= 65535

        return image

    def clean(self, image_id):
        nuclei_mask = np.load(test_nuclei + image_id + '.npy')
        cell_mask = np.load(test_cells + image_id + '.npy')
        image_path = '../input/hpa-single-cell-image-classification/test/' + image_id
        red = self.load_image(image_path + '_red.png')
        blue = self.load_image(image_path + '_blue.png')
        yellow = self.load_image(image_path + '_yellow.png')

        values = [1.0] * cell_mask.max()

        for cleaner in self.cleaners:
            values = cleaner(nuclei_mask, cell_mask, red, blue, yellow, values)
        
        return values

In [12]:
def solve_data_cleaner(x):
    dc = DataCleaner()
    v = dc.clean(x)
    return x, v

In [13]:
image_names = [x.rsplit('/',1)[1][:-8] for x in mt]
image_values = []

start_time = time.time()
pool = mp.Pool(CPU_COUNT)
    
for x in image_names:
    pool.apply_async(solve_data_cleaner, args=(x,), callback=image_values.append)

pool.close()    
pool.join()

elapsed_time = time.time() - start_time
print(elapsed_time / 60.0)

4.230285461743673


In [14]:
border_and_garbage_value = {}
for ID, vals in image_values:
    
    for i in range(len(vals)):
        border_and_garbage_value[ID + '_' + str(i)] = vals[i]

In [15]:
class ImageMetadata():
    def __init__(self, image_name, data_directory):
        self.image_name = image_name
        self.image_path = data_directory + image_name
    
    def __repr__(self):
        return self.image_path + ' - ' + str(self.image_labels)
    
class HPACellDataset(Dataset):
    
    def __init__(self, image_metadata, channels_num=4):
        self.image_metadata = image_metadata
        self.transforms = transforms.Compose([transforms.ToTensor()])

        if channels_num == 3:
            self.channels = ['_red', '_green', '_blue']
        elif channels_num == 4:
            self.channels = ['_red', '_green', '_blue', '_yellow']
        else:
            raise Exception('unsupported channels')

    def __len__(self):
        return len(self.image_metadata)
            
    def __getitem__(self, index):

        image_metadata = self.image_metadata[index]

        channels = []

        for channel in self.channels:

            image = cv2.imread(image_metadata.image_path + channel + '.png', cv2.IMREAD_UNCHANGED)
            image = image.astype(np.float32) / 65535.0
            channels.append(image)
        
        image = np.dstack(channels)

        return self.transforms(image), index

def generate_image_id_to_image_names_dictionary(data_directory):

    image_id_to_image_names_dictionary = {}
    
    image_names = set(image.rsplit('_',1)[0] for image in os.listdir(data_directory))

    for image_name in image_names:
        image_id = image_name.rsplit('_', 1)[0]

        image_id_list = image_id_to_image_names_dictionary.get(image_id, [])
        image_id_list.append(image_name)
        image_id_to_image_names_dictionary[image_id] = image_id_list

    return image_id_to_image_names_dictionary

def get_test_loader(test_dir, batch_size = 4, num_workers = CPU_COUNT, in_channels=4):
    image_id_to_image_name = generate_image_id_to_image_names_dictionary(test_dir)

    test_images = [row[0] for row in csv.reader(open('../input/hpa-single-cell-image-classification/sample_submission.csv', 'r')) if row[0] != 'ID']

    test_metadata = []
    cnt = 0

    for test_image in test_images:
        # TODO removed
        #if not test_image in image_id_to_image_name:
            #continue
        for test_cell in image_id_to_image_name[test_image]:
            test_metadata.append(ImageMetadata(test_cell, test_dir))
                                 
    dataset = HPACellDataset(test_metadata, in_channels)
    test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)
    
    return test_loader, dataset

In [16]:
# TODO fix batch size? put 8 for b0 on GPU
test_loader, dataset = get_test_loader(cells_512,8,CPU_COUNT, in_channels=4)

In [17]:
def merge_solution_values(solution_values_list, solution_values_weights):
    result_solution_values = SolutionValues()

    for i in range(len(solution_values_list)):
        solution_values = solution_values_list[i]
        weight = solution_values_weights[i]

        for ID, num, label in solution_values.values:
            value = solution_values.get_value(ID, num, label) * weight
            result_solution_values.add_value(ID, num, label, value)
    
    return result_solution_values

In [18]:
class SolutionValues():
    def __init__(self):
        self.values = {}
    
    def add_value(self, ID, num, label, value):
        key = (ID, num, label)
        self.values[key] = self.values.get(key, 0) + value
    
    def get_value(self, ID, num, label):
        key = (ID, num, label)
        return self.values[key]

    def weight_cells_per_image(self, cell_weight, image_weight, border_and_garbage_value):
        temp_values = {}

        for ID, num, label in self.values:
            if label in [11, 18] or border_and_garbage_value[ID + '_' + str(num)] != 1.0:
                continue

            key = (ID, label)
            value = self.values[(ID, num, label)]

            if not key in temp_values:
                temp_values[key] = []

            temp_values[key].append(value)

        for ID, num, label in self.values:
            key = (ID, label)
            if label in [11, 18] or not key in temp_values:
                continue

            value = self.values[(ID, num, label)] * cell_weight
            value += sum(temp_values[key]) / len(temp_values[key]) * image_weight

            self.values[(ID, num, label)] = value

    def calculate_negatives(self):
        temp_values = {}

        for ID, num, label in self.values:
            key = (ID, num)
            value = self.values[(ID, num, label)]

            if not key in temp_values:
                temp_values[key] = []
            
            temp_values[key].append(value)
        
        for ID, num in temp_values:
            key = (ID, num)
            values = temp_values[(ID, num)]
            
            self.values[(ID, num, 18)] = 1.0 - max(values)
    
    def weight_border_and_garbage_images(self, border_and_garbage_value):
        for ID, num, label in self.values:
            self.values[(ID, num, label)] = self.values[(ID, num, label)] * border_and_garbage_value[ID + '_' + str(num)]
        
    def get_values_per_image(self):
        values_per_image = {}

        for ID, num, label in self.values:
            key = (ID, num)
            value = self.values[(ID, num, label)]

            if not key in values_per_image:
                values_per_image[key] = []

            values_per_image[key].append((label, value))  

        return values_per_image
    
    def to_submission_file(self, sample_submission_path, submission_path):
        results = {}

        d = self.get_values_per_image()

        for x in d:
            s = []    
            image_name = x[0] + '_' + str(x[1])

            for y in d[x]:
                s.append(' '.join([str(y[0]), str(y[1]), mask_encodings[x[0]][x[1]]]))
            s = ' '.join(s)
            
            t = results.get(x[0], [])
            t.append(s)
            results[x[0]] = t

        with open(sample_submission_path, 'r') as file:
            reader = csv.reader(file)
            
            missed = 0

            with open(submission_path, 'w') as subf:
                for row in reader:
                    if row[0] in results:
                        tmp = ' '.join(results[row[0]])
                        print(f'{row[0]},{row[1]},{row[2]},{tmp}', file=subf)
                    else:
                        #print('SUCCESS!')
                        missed += 1
                        print(','.join(row), file=subf)
            
            print("MISSED:", missed)

In [19]:
def get_efficient_net_b0(state_dict_path):
    net = EfficientNet.from_name('efficientnet-b0', in_channels=4, num_classes=18, image_size=512)
    net._fc = nn.Sequential(nn.Linear(1280, 18), nn.Sigmoid())

    if torch.cuda.is_available():
        net.load_state_dict(torch.load(state_dict_path))
        net.to(device)
    else:
        net.load_state_dict(torch.load(state_dict_path ,map_location=device))

    net.eval()
    
    return net

def get_efficient_net_b4(state_dict_path):
    net = EfficientNet.from_name('efficientnet-b4', in_channels=4, num_classes=18, image_size=512)
    net._fc = nn.Sequential(nn.Linear(1792, 18), nn.Sigmoid())

    if torch.cuda.is_available():
        net.load_state_dict(torch.load(state_dict_path))
        net.to(device)
    else:
        net.load_state_dict(torch.load(state_dict_path ,map_location=device))
    
    net.eval()

    return net

In [20]:
def evaluate_net(net, test_loader, dataset):
    net.eval()

    solution_values = SolutionValues()

    for i, data in enumerate(test_loader, 0):

        inputs, indices = data
        inputs = inputs.to(device)
        outputs = net(inputs)

        for batch_index in range(len(indices)):
            dataset_index = indices[batch_index].item()
            metadata = dataset.image_metadata[dataset_index]

            image_name = metadata.image_name
            splitter = image_name.rsplit('_', 1)
            ID, num = splitter[0], int(splitter[1])

            for label in range(18):

                value = outputs[batch_index][label].item()
                solution_values.add_value(ID, num, label, value)
    
    return solution_values

In [21]:
#b0 RESIZE ensemble
net = get_efficient_net_b0('../input/cropytorchmodels/b0-resize-epoch-12-finetuned.pt')
solution_values1 = evaluate_net(net, test_loader, dataset)

net = get_efficient_net_b0('../input/cropytorchmodels/b0-resize-epoch-13-finetuned.pt')
solution_values2 = evaluate_net(net, test_loader, dataset)

net = get_efficient_net_b0('../input/cropytorchmodels/b0-resize-epoch-14-finetuned.pt')
solution_values3 = evaluate_net(net, test_loader, dataset)

net = get_efficient_net_b0('../input/cropytorchmodels/b0-resize-epoch-15-finetuned.pt')
solution_values4 = evaluate_net(net, test_loader, dataset)

solution_values_resize_ensemble = merge_solution_values([solution_values1, solution_values2, solution_values3, solution_values4], [0.20, 0.30, 0.30, 0.20])

In [22]:
net = get_efficient_net_b0('../input/cropytorchmodels/b0-resize-and-pad-epoch-12-finetuned.pt')
solution_values1 = evaluate_net(net, test_loader, dataset)

net = get_efficient_net_b0('../input/cropytorchmodels/b0-resize-and-pad-epoch-13-finetuned.pt')
solution_values2 = evaluate_net(net, test_loader, dataset)

solution_values_resize_and_pad_ensemble = merge_solution_values([solution_values1, solution_values2], [0.5, 0.5])

In [23]:
net = get_efficient_net_b0('../input/cropytorchmodels/b0-old.pt')
solution_values_old_b0 = evaluate_net(net, test_loader, dataset)

In [24]:
solution_values = merge_solution_values([solution_values_resize_ensemble, solution_values_resize_and_pad_ensemble, solution_values_old_b0], [0.40, 0.40, 0.20])

In [25]:
solution_values.calculate_negatives()
solution_values.weight_border_and_garbage_images(border_and_garbage_value)
solution_values.weight_cells_per_image(0.7, 0.3, border_and_garbage_value)

In [26]:
SAMPLE_SUBMISSION = '../input/hpa-single-cell-image-classification/sample_submission.csv'
SUBMISSION = '../working/submission.csv'
solution_values.to_submission_file(SAMPLE_SUBMISSION, SUBMISSION)

MISSED: 1
